In [1]:
!pip -q install langchain openai tiktoken

In [13]:
import urllib

use_long_text = True

if use_long_text:
  url = "https://raw.githubusercontent.com/mauricio-seiji/Dataset-news-articles-pt-br/main/ciencia%20e%20tecnologia/30.txt"
else:
  url = "https://raw.githubusercontent.com/mauricio-seiji/Dataset-news-articles-pt-br/main/ciencia%20e%20tecnologia/22.txt"

news_article = urllib.request.urlopen(url).read().decode("utf-8")
#

In [3]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter

model_name = "gpt-3.5-turbo"

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    model_name=model_name
)

texts = text_splitter.split_text(news_article)

docs = [Document(page_content=t) for t in texts]
print(len(docs))

2


In [21]:
# fetch the API key from the environment
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY)



In [7]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name=model_name)

C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [18]:
from langchain.prompts import PromptTemplate

prompt_template = """Write a concise summary of the following:

{text}

CONSCISE SUMMARY IN HINDI:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

In [19]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens = num_tokens_from_string(news_article, model_name)
print(num_tokens)

4688


In [20]:

from langchain.chains.summarize import load_summarize_chain
import textwrap
from time import monotonic


gpt_35_turbo_max_tokens = 4097
verbose = True

if num_tokens < gpt_35_turbo_max_tokens:
  chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt, verbose=verbose)
else:
  chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=prompt, combine_prompt=prompt, verbose=verbose)

start_time = monotonic()
summary = chain.run(docs)

print(f"Chain type: {chain.__class__.__name__}")
print(f"Run time: {monotonic() - start_time}")
print(f"Summary: {textwrap.fill(summary, width=100)}")



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:

Como a fruta mais fedida do mundo pode ajudar a gerar energia para seu telefone

Elas são o coração da tecnologia portátil moderna. As baterias de íon de lítio transformaram nossa capacidade de armazenar e transportar energia e, por sua vez, revolucionaram os dispositivos que usamos.

Comercializadas pela Sony em 1991, quando a empresa buscava uma solução para a duração limitada da bateria de suas câmeras de vídeo portáteis, elas fornecem energia a muitos dos gadgets que usamos hoje - de smartphones e laptops a escovas de dente elétricas e aspiradores de pó de mão. No fim do ano passado, os três cientistas por trás de sua invenção ganharam o Prêmio Nobel de Química por possibilitar essa revolução técnica.

E nossa necessidade por elas só tende a crescer. Os veículos elétricos dependem de baterias de íon-lítio como um substituto para os